# Lab 3 Advanced

## Linear Regression

### Model

Linear model: $y_i=\beta_0+\beta_1 x_{i1}+\dots+\beta_p x_{ip}+\epsilon$ for $i=1,\dots,n$

Matrix notation: $y=X\beta+\epsilon$

- $y$ - response variable
- $x_1, \dots, x_p$ - set of $p$ regressors
- $\epsilon$ - noise

Approximation: $\hat{y}=X\hat{\beta}$

### Ordinary Least Squares

Goal: Find values for $\beta$ that "best" fit the data

Question: What's the definition of "best"?

Error in predictions is the difference between the actual value and the predicted value: $y-\hat{y}$
![Image](https://i1.wp.com/statisticsbyjim.com/wp-content/uploads/2017/04/residuals.png?resize=300%2C186&ssl=1)

Squaring the difference accounts for overprediction and underprediction: $(y-\hat{y})^2$
![image](https://miro.medium.com/max/628/1*uBnjPy5o59FfkkMEJL0Nqw.jpeg)

The motivation behind OLS is minimizing the sum of squared errors: $\hat{\beta} = \underset{\beta}{\operatorname{argmin}} ||y-\hat{y}||_2^2$

OLS is BLUE.

### Assumptions of OLS

- Linearity - $E[y]=X\beta$
- Strict exogeneity - $E[\epsilon|X]=0$
- No perfect multicollinearity - Regressors can't be linearly dependent, X has full rank, $\Pr[\text{rank}(X)=p]=1$
- Independent errors
- Homoscedasticity - $E[\epsilon_i^2|X]=\sigma^2$
- No autocorrelation - $E[\epsilon_i\epsilon_j|X]=0$ for $i\neq j$

### Solving OLS

Goal: Minimize our objective $||y-\hat{y}||_2^2$

Idea: Take the derivative, set equal to 0, and solve for $\hat{\beta}$

\begin{align*}
||y-\hat{y}||_2^2 &= (y-\hat{y})^T (y-\hat{y}) \\
&= (y-X\hat{\beta})^T (y-X\hat{\beta}) \\
&= y^Ty - \hat{\beta}^TX^ty - y^TX\hat{\beta} + \hat{\beta}^TX^TX\hat{\beta} \\
&= y^Ty - 2\hat{\beta}^TX^ty + \hat{\beta}^TX^TX\hat{\beta} \\
\\
\nabla_\beta ||y-\hat{y}||_2^2 &= -2X^Ty + 2X^TX\hat{\beta} \\
\\
-2X^Ty + 2X^TX\hat{\beta} &= 0 \\
\Rightarrow X^TX\hat{\beta} &= X^Ty \\
\Rightarrow \hat{\beta} &= (X^TX)^{-1}X^Ty
\end{align*}

In [1]:
import sklearn as sk
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

## Implementing OLS

### Generate a dataset

Generate 1000 samples of $X_1 \sim U(0, 10)$

Generate 1000 samples of $X_2 \sim U(-20, -10)$

Generate $y = 2 + 10X_1 - 5X_2 + \epsilon$ where $\epsilon \sim N(0, 1)$

In [2]:
# Insert code here

In [14]:
n_samples = 1000
X = np.vstack((np.random.uniform(0, 10, n_samples), np.random.uniform(-20, -10, n_samples))).T
print(X.shape)
# print(X)
y = 2 + 10*X[:, 0]-5*X[:, 1] + np.random.normal(0, 1, n_samples)
# print(y)

(1000, 2)


### Solve for $\beta$

1. Use the closed form solution derived above (HINT: you need to add a column of ones)

2. Use sklearn's linear regression

Check that estimated coefficients from methods 1 and 2 are the same and match the data generating process.

Why do we need to add a column of ones for method 1? What happens if we don't add any noise ($\epsilon$) to our data?

In [4]:
# Insert code here

In [5]:
X_matrix = np.vstack((np.ones((X.shape[0])), X.T)).T
print('X', X)
print('X_matrix shape', X_matrix.shape )
print(X_matrix)

print('\nwith ones column:')
comput =(X_matrix.T.dot(X_matrix)).dot(X_matrix.T).dot(y)
print((X_matrix.T.dot(X_matrix)).dot(X_matrix.T).dot(y))
print(np.linalg.inv(X_matrix.T.dot(X_matrix)).dot(X_matrix.T).dot(y))
print(sk.linear_model.LinearRegression().fit(X, y).coef_)

print('without ones column:')
print(np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y))
print(sk.linear_model.LinearRegression().fit(X, y).coef_)

X [[  5.48813504 -14.07119729]
 [  7.15189366 -19.89936304]
 [  6.02763376 -15.24173804]
 ...
 [  9.38412022 -15.19892193]
 [  2.28646551 -13.56135963]
 [  6.77141144 -14.98226869]]
X_matrix shape (1000, 3)
[[  1.           5.48813504 -14.07119729]
 [  1.           7.15189366 -19.89936304]
 [  1.           6.02763376 -15.24173804]
 ...
 [  1.           9.38412022 -15.19892193]
 [  1.           2.28646551 -13.56135963]
 [  1.           6.77141144 -14.98226869]]

with ones column:
[ 3.22237853e+10  1.65691534e+11 -4.96822341e+11]
[ 1.81762706  9.99255568 -5.01306829]
[ 9.99255568 -5.01306829]
without ones column:
[10.03025659 -5.11836983]
[ 9.99255568 -5.01306829]


In [6]:
# Insert answers here

A column of ones is needed in order to properly estimate $\beta_0$. When we go from $y=\beta_0+\beta_1 X_1+\beta_2 X_2$ to $y=\beta X$, we are adding in the ones in between as $y=\beta_0 1 + \beta_1 X_1 + \beta_2 X_2$.

If we don't add noise to our data, our estimated coefficients will be the same as the coefficients we used to generate our data.

## Regularization

Regularization helps prevent overfitting and increases the generalizability of your model. By adding regularization, we are reducing variance at the expense of bias in our model. The two most common forms of regularization for linear regression are ridge regression and LASSO. These two regularization methods add a L2 or L1 penalty term to the objective, respectively.

Ridge regression: $||y-\hat{y}||_2^2 + \lambda||\beta||_2^2$

LASSO: $||y-\hat{y}||_2^2 + \lambda||\beta||_1$

Ridge regression is able to shrink all coefficients towards 0 while LASSO can set some coefficients towards 0. Because of this, LASSO is also able to perform feature selection. The last section of this blog post explains this concept visually. https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b

In order to effectively use regularization, all the regressors ($X$) must be standardized so coefficients can be compared with each other and penalized accordingly. For both of these methods, the regularization parameters needs to be tuned.

Why does standardization change the coefficients but not the $R^2$ score of OLS?

Why does standarization change the coefficients and the $R^2$ score for LASSO?

Derive the closed form solution to ridge regression (HINT: you should get $\hat{\beta}=(X^TX+\lambda I)^{-1}X^Ty$)

In [7]:
# Insert answers here

For OLS, the coefficients will be the inverse transformation of the standardization. The $R^2$ score (and any measure of fit) stays the same because the predicted values stay the same. For LASSO, the coefficents change because the coefficents no longer depend on the scale of the data which can lead to different coefficients being set to 0, changing the predicted value and the $R^2$ score.

In [8]:
# Insert derivation here

You can augment $X$ and $y$ and solve as previously shown. More details are given in the 2nd answer of https://stats.stackexchange.com/questions/69205/how-to-derive-the-ridge-regression-solution

## Effects of collinearity

Using the dataset from the previous section, add another variable $X_3 = .5 X_1 + .5 X_2 + \epsilon$ where $\epsilon \sim N(0, 0.1)$

Generate $y = 2 + 10X_1 - 5X_2 + 7X_3 + \epsilon$ where $\epsilon \sim N(0, 1)$

Do not standardize your variables

In [9]:
# Insert code here

In [10]:
X_col = np.vstack((X.T, .5*X[:, 0] + .5*X[:, 1] + np.random.normal(0, .1, X.shape[0]))).T
y_col = y+7*X_col[:, 2]

Use OLS to estimate the coefficients. How close are they to $\beta$?

Use ridge regression with the default hyperparameters to estimate the coefficients. How close are they to $\beta$?

Use LASSO with the default hyperparameters to estimate the coefficients. How close are they to $\beta$?

Explain the behavior of these three methods.

What would happen if $X_3 = .5X_1 + .5X_2$?

In [11]:
# Insert code here

In [18]:
print(sk.linear_model.LinearRegression().fit(X_col, y_col).coef_)
print(sk.linear_model.Ridge().fit(X_col, y_col).coef_)
print(sk.linear_model.Lasso().fit(X_col, y_col).coef_)

[ 9.83554888 -5.17063487  7.31459695]
[10.05867043 -4.94495873  6.86513126]
[13.36695253 -1.39451947  0.        ]


In [13]:
# Insert answers here

The OLS and ridge regression estimates are close to the true values.  Ridge regression splits the contribution of $X_3$ between $X_1$ and $X_2$. If we changed $X_3$ as above, then OLS would not longer be valid as a result of multicollinearity.

### Image sources
- https://statisticsbyjim.com/glossary/ordinary-least-squares/
- https://medium.com/@saahil1292/machine-learning-102-linear-regression-ordinary-least-squares-ols-correlation-and-analysis-of-7d53751ea9f4